In [1]:
import tensorflow as tf

In [2]:
x = tf.Variable(3, name='x')
y = tf.Variable(4, name='y')
f = x*x*y + y + 2

In [3]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)
sess.close()

42


In [4]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
    print(result)

42


In [5]:
init = tf.global_variables_initializer()

In [6]:
with tf.Session() as sess:
    init.run()
    print(f.eval())

42


In [7]:
sess = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)
sess.close()

42


In [8]:
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [9]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)
x2.graph is graph

True

In [10]:
x2.graph is tf.get_default_graph()

False

In [11]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

with tf.Session() as sess:
    print(y.eval())
    print(z.eval())

10
15


In [12]:
with tf.Session() as sess:
    y_result, z_result = sess.run([y, z])
    print(y_result, z_result)

10 15


In [13]:
import numpy as np
from sklearn.datasets import fetch_california_housing

In [14]:
housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

with tf.Session() as sess:
    theta_value = theta.eval()

In [15]:
print(theta_value)

[[-3.6894890e+01]
 [ 4.3661433e-01]
 [ 9.4453208e-03]
 [-1.0704148e-01]
 [ 6.4345831e-01]
 [-3.9632569e-06]
 [-3.7880042e-03]
 [-4.2093179e-01]
 [-4.3400639e-01]]


In [16]:
from sklearn.preprocessing import StandardScaler

In [17]:
std_scaler = StandardScaler()
scaled_housing_data_plus_bias = std_scaler.fit_transform(housing_data_plus_bias)

In [18]:
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

learning_rate = 0.01

X = tf.placeholder(dtype=tf.float32, shape=(None, n + 1), name='X')
y = tf.placeholder(dtype=tf.float32, shape=(None, 1), name='y')

def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)
    indices = np.random.randint(m, size=batch_size)
    X_batch = scaled_housing_data_plus_bias[indices]
    y_batch = housing.target.reshape(-1, 1)[indices]
    return X_batch, y_batch
    
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

In [19]:
mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [20]:
n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

In [21]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        
    best_theta = theta.eval()

In [22]:
file_writer.close()

In [23]:
best_theta

array([[ 0.9045429 ],
       [ 0.82172453],
       [ 0.11330824],
       [-0.21355605],
       [ 0.32225588],
       [-0.00730818],
       [ 0.00515022],
       [-0.88384354],
       [-0.85685915]], dtype=float32)

In [24]:
A = tf.placeholder(tf.float32, shape=(None, 3))
B = A + 5
with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict={A: [[1, 2, 3]]})
    B_val_2 = B.eval(feed_dict={A: [[4, 5, 6], [7, 8, 9]]})

In [25]:
print(B_val_1)

[[6. 7. 8.]]


In [26]:
print(B_val_2)

[[ 9. 10. 11.]
 [12. 13. 14.]]


In [27]:
saver = tf.train.Saver()
with tf.Session() as sess:
    saver.restore(sess, '/tmp/my_model_mini_batch.ckpt')
    print(best_theta)

W0627 10:27:58.674802 139724982327104 deprecation.py:323] From /home/sean/venv/py37/lib/python3.7/site-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


[[ 0.9045429 ]
 [ 0.82172453]
 [ 0.11330824]
 [-0.21355605]
 [ 0.32225588]
 [-0.00730818]
 [ 0.00515022]
 [-0.88384354]
 [-0.85685915]]
